<a href="https://colab.research.google.com/github/yoonandmoon/GS_team/blob/cleansing_yschoi/231026_GS_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_8%EC%9B%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_data08.csv')

df2 = df[['title','date','content']]
df2['content'] = df['content'].str.replace('\n',' ').replace('\t',' ').replace('\r', ' ')

<ipython-input-2-67a978c0538e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['content'] = df['content'].str.replace('\n',' ').replace('\t',' ').replace('\r', ' ')


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.3 MB/s eta 0:00:00


In [ ]:
pip install mecab-ko-msvc mecab-ko-dic-msvc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 MB 7.6 MB/s eta 0:00:00


In [ ]:
import MeCab
import pandas as pd
import re

# Mecab 인스턴스 생성
m = MeCab.Tagger()

# 'title' 열의 결과를 'titl' 열에 저장
df2['titl'] = df2['title'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', x))

# 'content' 열의 결과를 'cont' 열에 저장
df2['cont'] = df2['content'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', str(x)))

def extract_words(text):
    # 텍스트를 Mecab을 사용하여 토큰화합니다.
    tokens = m.parse(str(text)).split('\n')

    # 선택된 단어를 저장할 리스트
    selected_words = []

    for token in tokens:
        if len(token) >= 2:
            # 각 토큰을 탭(\t)으로 분리하여 형태소와 품사 정보를 얻습니다.
            parts = token.split('\t')
            if len(parts) == 2:
                word, pos = parts
                pos = pos.split(',')[0]  # 품사 정보에서 첫 번째 부분만 사용

                # 선택 조건 (명사, 동사, 형용사)을 확인하고 선택된 단어 리스트에 추가
                if len(word) >= 2 and pos in ['NNG', 'VV', 'VA']:
                    selected_words.append(word)

    # 선택된 단어를 공백으로 구분된 문자열로 반환합니다.
    return ' '.join(selected_words)

# 'title' 및 'content' 열에 대해 처리
for i, row in df2.iterrows():
    df2.at[i, 'titl'] = extract_words(row['title'])
    df2.at[i, 'cont'] = extract_words(row['content'])

df3 = df2[['date','titl','cont']]


<ipython-input-5-df7b657af641>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['titl'] = df2['title'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', x))


In [ ]:
from collections import Counter

words_list = df3['cont'].str.split()

all_words = [word for words in words_list for word in words]

word_counts = Counter(all_words)

most_common_words = word_counts.most_common(50)
print("가장 많이 나온 상위 50개 단어:")
for word, count in most_common_words:
    print(f"{word}: {count}")


stop_word = ['기자','지난해','올해',"따르",'건설','기자']

def preprocess(text):
  text = text.split()
  text = [i for i in text if i not in stop_word]
  return text

def preprocess(text):
  text = text.split()
  text = [i for i in text if i not in stop_word]
  return text

def make_tokens(df):
  df['tokens'] = ' '
  for i, row in df.iterrows():
    if i%100==0:
      print(i,'/',len(df))
    token = preprocess(df['cont'][i])
    df['tokens'][i] = ' '.join(token)
  return df

df4 = make_tokens(df3)



가장 많이 나온 상위 50개 단어:
건설: 6293
아파트: 3955
국토: 2927
영업: 2726
처분: 2700
정지: 2482
사고: 2208
현장: 2077
사업: 2028
공사: 1731
안전: 1603
시공: 1508
철근: 1475
검단: 1393
주택: 1360
지하: 1326
장관: 1318
붕괴: 1315
주차장: 1283
단지: 1279
점검: 1265
구조: 1211
업체: 1202
조사: 1178
관련: 1154
분양: 1154
가구: 1136
시장: 1070
추진: 1067
누락: 1057
결과: 1043
기술: 1030
기업: 1020
기자: 1000
계획: 941
발생: 938
설계: 918
예정: 915
문제: 913
진행: 901
무량: 894
경우: 868
투자: 858
건설사: 835
개발: 811
가능: 776
정부: 773
관리: 758
수주: 706
산업: 702
0 / 1258
100 / 1258
200 / 1258
300 / 1258


<ipython-input-7-10429cdefc54>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = ' '
<ipython-input-7-10429cdefc54>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'][i] = ' '.join(token)


400 / 1258
500 / 1258
600 / 1258
700 / 1258
800 / 1258
900 / 1258
1000 / 1258
1100 / 1258
1200 / 1258


In [ ]:
# 이후 불용어 확인 후 반복 이때는 불용어 처리시 이 코드 사용
#from collections import Counter

#words_list = df4['tokens'].str.split()

#all_words = [word for words in words_list for word in words]

#word_counts = Counter(all_words)

#most_common_words = word_counts.most_common(50)
#print("가장 많이 나온 상위 50개 단어:")
#for word, count in most_common_words:
    #print(f"{word}: {count}")

#날짜만 추출
df4['date'] = df4['date'].str.split(' ').str[0]
print(df4)

#csv파일 만들기
df4.to_csv('8월_전처리.csv', index=False)

            date                          titl  \
0     2023-08-01     사설 철근 빼먹 아파트 국민 안전 이권 카르텔   
1     2023-08-01                도색 분노 보강 공사 비용   
2     2023-08-01            데스크 칼럼 부실 공사 부르 고령   
3     2023-08-01                현장 기자 발주 관심 변명   
4     2023-08-01  지방 광역시 공급 올해 수도 추월 회복세 지방 확산   
...          ...                           ...   
1253  2023-09-02                  가전 박람회 혁신 제품   
1254  2023-09-02                      토막 설주 이때   
1255  2023-09-02     현장 가전 주택 친환경 효율 전자 기술 총망라   
1256  2023-09-02                      영상 가격 비즈   
1257  2023-09-02                     전자 만드 살펴보   

                                                   cont  \
0     국토 장관 지하 주차장 붕괴 사고 발생 검단 아파트 신축 공사장 모습 발주 아파트 ...   
1     앵커 이번 정부 발표 철근 누락 사실 아파트 주민 분노 설명 논의 보강 공사 반응 ...   
2     유통 산업 부장 고령화 실질 타격 실상 직시 다음 세대 얘기 나라 치부 단계 녹차 ...   
3     산업 기자 대통령 토지 주택 공사 무더기 아파트 사태 관련 아파트 지하 주차장 부실...   
4     지방 광역시 분양 예정 물량 올해 처음 수도 공급 물량 추월 전망 사진 일대 아파트...   
...                                          

<ipython-input-8-b4e6bc999c30>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['date'] = df4['date'].str.split(' ').str[0]
